$$\LARGE
\underline
{
\textbf
    {Problema 5}
}
$$

Determine los desplazamientos de los nodos y sus pendientes; las fuerzas en cada elemento y las
reacciones. Tome $E = 29x106\ psi$ (libras por pulgada cuadrada) e $I = 200\ pulgada^{4}$.

<center><img src="ImgProb05G02.png"></center>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import MEF as mef

In [2]:
E = 29e6  # psi
I = 200  # in4
L_barra = 30  # ft
w = 200  # lb/ft

In [3]:
MN = np.array([[0,0],[15,0],[30,0]])
MC = np.array([[0,1],[1,2]])

In [4]:
Nn, glxn = MN.shape  # cantidad de nodos, grados de lib x nodo
Ne, Nnxe = MC.shape  # cantidad de elementos, numero de nodos x elemento

In [5]:
L = L_barra/Ne  # Longitud de cada elemento

In [6]:
s = np.array([0,1,4])  # d y phi conocidos
r = mef.vector_complemento(s,MN,glxn)

In [7]:
MR_elem = np.array([[12,6*L,-12,6*L],[6*L,4*L**2,-6*L,2*L**2],[-12,-6*L,12,-6*L],[6*L,2*L**2,-6*L,4*L**2]])
MR_elem = (E*I/L**3)*MR_elem

In [8]:
Ke = {}  # diccionario para acumular todas las K_elementales
MR = np.zeros([glxn*Nn,glxn*Nn])
archivo = 'Matrices.txt'  # Creo archivo para guardar todas las matrices elementales
with open(archivo,'w') as f:  # la f es como un alias apra el archivo que acabo de abrir
    f.write('Matrices Elementales\n ===============')
    
for e in range(Ne):
    Ke[e] = MR_elem
    fe = np.abs(Ke[e].max())
    with open('Matrices.txt','a') as f:  # 'a' de agregar
        f.write(f'\nelemento {e}, fe ={fe:4e}\n')
        f.write(f'{Ke[e]/fe}\n')

    for i in range(Nnxe):
        rangoi = np.linspace(i*glxn,(i+1)*glxn-1,glxn).astype(int)
        rangoni = np.linspace(MC[e,i]*glxn,(MC[e, i]+1)*glxn-1,glxn).astype(int)
        for j in range(Nnxe):
            rangoj = np.linspace(j*glxn,(j+1)*glxn-1,glxn).astype(int)
            rangonj = np.linspace(MC[e,j]*glxn,(MC[e, j]+1)*glxn-1,glxn).astype(int)
            MR[np.ix_(rangoni,rangonj)] += Ke[e][np.ix_(rangoi,rangoj)]

fe = np.abs(MR.max())
with open('Matrices.txt','a') as f:  # 'a' de agregar
    f.write(f'\nMatriz Global, fe ={fe:4e}\n')
    f.write(f'{MR/fe}\n')

$$
$$

$ \Large \underline
{\textbf{Viga con carga uniformemente distribuida en 2 nodos}}:
$

$ \Large
\begin{matrix}
f_{1y}&m_{1}&f_{2y}&m_{2}\\
\frac{-wL}{2}&\frac{-wL^2}{12}&\frac{-wL}{2}&\frac{wL^2}{12}
\end{matrix}
$

In [29]:
f = np.array([-w*L/2,-w*L**2/12,-w*L/2,0,-w*L/2,w*L**2/12]).reshape([-1,1])  # fuerzas y momentos externos
# el momento en el nodo del medio lo considero nulo, se cancela porque cada partición genera un momento opuesto
d = np.zeros(Nn*glxn).reshape([-1,1])  # vector que serán los desplazamientos y angulos

In [10]:
d[r] = np.linalg.solve(MR[np.ix_(r,r)],f[r])  # saco los desplazamientos y angulos que eran incognitos

In [27]:
Ft = MR.dot(d)  # Fuerzas totales

In [28]:
Fr = Ft - f  # Fuerzas de reacción

In [25]:
np.set_printoptions(formatter={"float": lambda x: "%0.2f"% (x)})
print('\033[4m'+f'Matriz global'+'\033[0m'+':\n\033[93m'+ f'U =\n {MR/MR.max()} \n'+'\033[0m')
print('\033[4m'+f'\nVector fuerzas y momentos' +'\033[0m'+':\n\033[93m'+f'F = \n{Fr}\n'+'\033[0m')
print('\033[4m'+f'Vector desplazamientos y angulos'+'\033[0m'+':\n\033[93m'+ f'U =\n {d*1e6} \n')

Matriz global:
U =
 [[0.01 0.05 -0.01 0.05 0.00 0.00]
 [0.05 0.50 -0.05 0.25 0.00 0.00]
 [-0.01 -0.05 0.01 0.00 -0.01 0.05]
 [0.05 0.25 0.00 1.00 -0.05 0.25]
 [0.00 0.00 -0.01 -0.05 0.01 -0.05]
 [0.00 0.00 0.05 0.25 -0.05 0.50]] 


Vector fuerzas y momentos:
F = 
[[2718.75]
 [14062.50]
 [-0.00]
 [0.00]
 [1781.25]
 [0.00]]

Vector desplazamientos y angulos:
U =
 [[0.00]
 [0.00]
 [-81.83]
 [-3.03]
 [0.00]
 [12.12]] 

